<a href="https://colab.research.google.com/github/katshalonskaya/Project-for-AoSN/blob/main/AoSN_zd_data_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
import pandas as pd
import numpy as np
import os
import itertools
!pip install networkx
import networkx as nx
from itertools import combinations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({'id':'1gcAfMXpoWA6iXBWCocrlbFOnkoyOgD7Q'})
downloaded.GetContentFile("cntch.csv",  mimetype='csv')
cntnt = pd.read_csv('cntch.csv')
downloaded = drive.CreateFile({'id':'1lvF-uxPP0DNFxwuFwMpwoPtm5wo6UUqr'})
downloaded.GetContentFile("prblmch.csv",  mimetype='csv')
prblm = pd.read_csv('prblmch.csv')
downloaded = drive.CreateFile({'id':'152oXJcgGKudEKCM2oPapQc4YXc7lpeDX'})
downloaded.GetContentFile("usrch.csv",  mimetype='csv')
usr = pd.read_csv('usrch.csv')

In [5]:
cntnt.pop("level1_id")
cntnt.pop("level2_id")
cntnt.pop("level3_id")
cntnt.pop("level4_id")

0       KPJMQebU0O24+NzlQ4udb2BXLlKV1Hte61+hV5Xb+oU=
1       KPJMQebU0O24+NzlQ4udb2BXLlKV1Hte61+hV5Xb+oU=
2       KPJMQebU0O24+NzlQ4udb2BXLlKV1Hte61+hV5Xb+oU=
3       KPJMQebU0O24+NzlQ4udb2BXLlKV1Hte61+hV5Xb+oU=
4       KPJMQebU0O24+NzlQ4udb2BXLlKV1Hte61+hV5Xb+oU=
                            ...                     
1329                                             NaN
1330                                             NaN
1331                                             NaN
1332                                             NaN
1333                                             NaN
Name: level4_id, Length: 1334, dtype: object

In [6]:
cntnt.pop("subject")

0       math
1       math
2       math
3       math
4       math
        ... 
1329     NaN
1330     NaN
1331     NaN
1332     NaN
1333     NaN
Name: subject, Length: 1334, dtype: object

In [7]:
cntnode = cntnt[['Unnamed: 0', 'difficulty', 'learning_stage']]
cntnode = cntnode.dropna()
cntnode.rename(columns = {'Unnamed: 0' : 'idcn'}, inplace = True) 
cntnode.head()

,idcn,difficulty,learning_stage
0,0,0.0,0.0
1,1,0.0,0.0
2,2,0.0,0.0
3,3,2.0,0.0
4,4,2.0,0.0


In [8]:
usr.pop('badges_cnt')
usr.pop('first_login_date_TW')
usr.pop('user_city')
usr.head()

,Unnamed: 0,uuid,gender,points,user_grade,has_teacher_cnt,is_self_coach,has_student_cnt,belongs_to_class_cnt,has_class_cnt
0,396,fUiuB7SPD9QYApc5GYqFs2+IdzYPILeKDL8HEdGc1ek=,1,104820,5,1,1,0,1,0
1,1313,25BBuM6hiBHNi3SIop1N3yVZcYwmSCrHoN5dnyt397I=,0,126082,5,1,1,0,1,0
2,2469,cYPUcxPHHzfyA3GTSkRL/IfXYWq5idkCOCDw3elQIOc=,0,3660,5,1,1,0,1,0
3,11426,oXChecbW08XhQ83WzOMPDOk+Gq5dGaVrqO/RiDPkjtQ=,1,1275,7,1,1,0,1,0
4,14438,4IEuyf0Q/ZcI814fUZCb2o72QEjynwbihuzd6AWSUZU=,0,69397,6,0,1,0,0,0


In [9]:
prblm.pop('total_sec_taken')
prblm.pop('total_attempt_cnt')
prblm.pop('used_hint_cnt')
prblm.head()

,Unnamed: 0,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,is_hint_used,level
0,3317,2019-06-02 19:15:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,rmdqcNIKqoQBDP6UuEnjhgIL6vm+qw6zm5NOsyeFI1s=,36,3,0,1,2
1,7415,2018-09-29 17:15:00 UTC,7d++UMRHGrnoRmVXey3umeKlOwA22bCuSMbVGO9QKII=,gwyAKrDd3r/aPElAq/97BWk1H4+bsSURhZI+O3WDwFY=,flfeUQqShTQd40KGf0leEUYD60rtQecY6MMGr4xdres=,4,1,0,1,0
2,10638,2018-10-16 16:45:00 UTC,9SgNlDjJycgGdjrzBt7AZBN6AFHupfn3ewBDfAWsNdQ=,RKkxBe+3Z33I75tZfZ8EY7LtVnN5QEfE8O0y6bPFCeo=,STZD5cyjUf3J2AETFRoMb+LREPAPoAY3Cc8b63jV3Qw=,2,1,1,0,0
3,14824,2019-06-03 20:00:00 UTC,4qTG7XqzmOWUpeIhEJ/eN05YJf0C9lxbxGPa7bUeGBQ=,bZml5gkJWB1WJFn9WFcAJB4dwFXSFI3h0uK0BwtBHzg=,MSas2P953+W2pAXMCsguOxNRn9VrA+etQEsGy8YUR7M=,12,1,1,1,0
4,20722,2019-05-12 09:45:00 UTC,QCNXGnKbcSS9WfPo0QmpjND7JQWh9RPbBdqiuEiJ9Lk=,5XOh9hPgSIw+4ArOqH8SRFzrFrG107tCCpwFQcpZ9T8=,DgKbAW6IXGzG5ed4FZrC4MPSTrb64l7Dxj2oJYLJfjo=,2,1,0,1,0


In [35]:
gnrl = pd.merge(prblm,usr,on="uuid", how="inner")
gnrl.rename(columns = {'Unnamed: 0_x' : 'idpr', 'Unnamed: 0_y' : 'idus'}, inplace = True) 
gnrl.head()

,idpr,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,is_hint_used,level,idus,gender,points,user_grade,has_teacher_cnt,is_self_coach,has_student_cnt,belongs_to_class_cnt,has_class_cnt
0,3317,2019-06-02 19:15:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,rmdqcNIKqoQBDP6UuEnjhgIL6vm+qw6zm5NOsyeFI1s=,36,3,0,1,2,59721,1,27211,5,0,1,0,0,1
1,61231,2019-05-29 21:00:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,rmdqcNIKqoQBDP6UuEnjhgIL6vm+qw6zm5NOsyeFI1s=,20,2,0,1,1,59721,1,27211,5,0,1,0,0,1
2,174857,2018-11-21 18:45:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,65YgA0ahggSdeaYBdRl82v/AO+pGzmZFqsi8TTDiP3Y=,JjWuxX+Dp4Y9s30BGgiO+CGjcpOSWuSA5WM3y8ObV88=,16,2,0,1,1,59721,1,27211,5,0,1,0,0,1
3,176442,2018-11-21 18:30:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,65YgA0ahggSdeaYBdRl82v/AO+pGzmZFqsi8TTDiP3Y=,DzBAuk08F2ttS0SAVU21HPRgQqbfbdyMDdjcPQEc9nY=,7,1,0,1,0,59721,1,27211,5,0,1,0,0,1
4,412577,2018-11-21 18:45:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,WzvugOYA7I7EAC0ty8LiMKIA+9T/CYrBJx5dG+/QcMw=,J2ruTHqIt7sQ8Gm0RBe3EFykqR8/sEnghHJjbVNfgJ0=,3,1,1,0,0,59721,1,27211,5,0,1,0,0,1


In [36]:
gnrl = pd.merge(gnrl,cntnt,on="ucid", how="inner")
gnrl.rename(columns = {'Unnamed: 0' : 'idcn'}, inplace = True) 
gnrl.head()

,idpr,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,is_hint_used,level,...,user_grade,has_teacher_cnt,is_self_coach,has_student_cnt,belongs_to_class_cnt,has_class_cnt,idcn,difficulty,learning_stage,content_name
0,3317,2019-06-02 19:15:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,rmdqcNIKqoQBDP6UuEnjhgIL6vm+qw6zm5NOsyeFI1s=,36,3,0,1,2,...,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
1,61231,2019-05-29 21:00:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,rmdqcNIKqoQBDP6UuEnjhgIL6vm+qw6zm5NOsyeFI1s=,20,2,0,1,1,...,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
2,1643806,2019-05-29 19:00:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,GUOHQNHJfPcaIimL5DYusewEe2AUmzy5VSsSeKiazPs=,6,1,0,1,0,...,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
3,1652167,2019-06-02 19:15:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,Y6SGgz5raRWUUbf2lFitqEmbik78y1l6zWmUh8mJE/Q=,32,3,0,1,2,...,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
4,3073631,2019-05-29 21:00:00 UTC,Dq4uCsNL9/pg7BcNMwuUB156IW9ELoXMZEBIxeBRhAs=,rzWpkZfVptJjhCtmA5xHgJIsAZQKyQ0mwHzaE2K/I9Y=,Q2dFHwqg5pmx4zpQ5RXOEl7VMz0bWjGwoGIAFY/dsSc=,21,1,1,0,1,...,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...


In [37]:
gnrl.pop("uuid")
gnrl.pop("timestamp_TW")
gnrl.pop("ucid")
gnrl.pop("upid")
gnrl.head()

,idpr,problem_number,exercise_problem_repeat_session,is_correct,is_hint_used,level,idus,gender,points,user_grade,has_teacher_cnt,is_self_coach,has_student_cnt,belongs_to_class_cnt,has_class_cnt,idcn,difficulty,learning_stage,content_name
0,3317,36,3,0,1,2,59721,1,27211,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
1,61231,20,2,0,1,1,59721,1,27211,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
2,1643806,6,1,0,1,0,59721,1,27211,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
3,1652167,32,3,0,1,2,59721,1,27211,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...
4,3073631,21,1,1,0,1,59721,1,27211,5,0,1,0,0,1,429,0.0,0.0,[Основной] Распознавание двух знаков после зап...


In [38]:
nodesus = gnrl[['idcn', 'difficulty', 'learning_stage', "idpr",	"problem_number",	"exercise_problem_repeat_session",	"is_correct",	"is_hint_used",	"level"]]
nodesus.head()
n = gnrl[['idcn', 'difficulty', 'learning_stage']]

In [39]:
gnrl2 = gnrl[['idcn', 'difficulty', 'learning_stage', "idus",	"gender",	"points",	"user_grade",	"has_teacher_cnt",	"is_self_coach",	"has_student_cnt",	"belongs_to_class_cnt",	"has_class_cnt"]]
gnrl2.head()

,idcn,difficulty,learning_stage,idus,gender,points,user_grade,has_teacher_cnt,is_self_coach,has_student_cnt,belongs_to_class_cnt,has_class_cnt
0,429,0.0,0.0,59721,1,27211,5,0,1,0,0,1
1,429,0.0,0.0,59721,1,27211,5,0,1,0,0,1
2,429,0.0,0.0,59721,1,27211,5,0,1,0,0,1
3,429,0.0,0.0,59721,1,27211,5,0,1,0,0,1
4,429,0.0,0.0,59721,1,27211,5,0,1,0,0,1


In [14]:
gnrl3 = gnrl[['idcn', "idus",]]
gnrl3.head()

,idcn,idus
0,429,59721
1,429,59721
2,429,59721
3,429,59721
4,429,59721


In [15]:
gnrl4 = gnrl3.groupby("idus")["idcn"].apply(list).agg(tuple).to_frame()
gnrl4.tail()

,idcn
idus,
65371,"(279, 279, 279, 279, 279)"
67824,"(143, 143, 143, 393, 393, 393, 393, 393, 393, ..."
68312,"(435, 435, 435, 435, 435, 435, 435, 435, 435, ..."
69378,"(402, 402, 402, 402, 402, 402, 402, 402, 402, ..."
72645,"(429, 429, 429, 429, 429, 454, 454, 454, 454, ..."


In [ ]:
from google.colab import files
gnrl4.to_csv('gnrl5.csv') 
files.download('gnrl5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Выгружено для удаления дубликатов, чтобы не перегружать базу

In [16]:
downloaded = drive.CreateFile({'id':'1CU4hKomZ6u90YEtguPwnfLJxEugssObw'})
downloaded.GetContentFile("gnrlusq.csv",  mimetype='csv')
gn = pd.read_csv('gnrlusq.csv')

In [17]:
gn.head()

,idus,idcn_1,idcn_2,idcn_3,idcn_4,idcn_5,idcn_6,idcn_7,idcn_8,idcn_9,...,idcn_116,idcn_117,idcn_118,idcn_119,idcn_120,idcn_121,idcn_122,idcn_123,idcn_124,idcn_125
0,396,382,534.0,379.0,373.0,600.0,384.0,36.0,466.0,385.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1313,282,344.0,325.0,328.0,638.0,345.0,467.0,350.0,352.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2469,578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11426,129,120.0,127.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14438,467,469.0,470.0,468.0,471.0,348.0,341.0,614.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def func_name(df):
  df = df.drop('idcn',1)
  rf = pd.DataFrame()
  tf = df
  for idpr_1 in df:
    tf = tf.drop(idpr_1,1)
    for idpr_2 in tf:
      d_to_ap = []
      for i in range(0,len(df)):
        if np.isnan(df[idpr_1][i]) == True or (df[idpr_1][i]) == 0:
          x = df[idpr_1][i]
        elif np.isnan(df[idpr_2][i]) == True or (df[idpr_2][i]) == 0:
          x = df[idpr_2][i]
        else:
          x = (df[idpr_1][i],df[idpr_2][i])
        d_to_ap.append(x)
      rf[idpr_1+', '+idpr_2] = d_to_ap

    print('Покончено с ячейкой',idpr_1)
  return rf

In [ ]:
edges=func_name(gn)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Покончено с ячейкой idcn_1
Покончено с ячейкой idcn_2
Покончено с ячейкой idcn_3
Покончено с ячейкой idcn_4
Покончено с ячейкой idcn_5
Покончено с ячейкой idcn_6
Покончено с ячейкой idcn_7
Покончено с ячейкой idcn_8
Покончено с ячейкой idcn_9
Покончено с ячейкой idcn_10
Покончено с ячейкой idcn_11
Покончено с ячейкой idcn_12
Покончено с ячейкой idcn_13
Покончено с ячейкой idcn_14
Покончено с ячейкой idcn_15
Покончено с ячейкой idcn_16
Покончено с ячейкой idcn_17
Покончено с ячейкой idcn_18
Покончено с ячейкой idcn_19
Покончено с ячейкой idcn_20
Покончено с ячейкой idcn_21
Покончено с ячейкой idcn_22
Покончено с ячейкой idcn_23
Покончено с ячейкой idcn_24
Покончено с ячейкой idcn_25
Покончено с ячейкой idcn_26
Покончено с ячейкой idcn_27
Покончено с ячейкой idcn_28
Покончено с ячейкой idcn_29
Покончено с ячейкой idcn_30
Покончено с ячейкой idcn_31
Покончено с ячейкой idcn_32
Покончено с ячейкой idcn_33
Покончено с ячейкой idcn_34
Покончено с ячейкой idcn_35
Покончено с ячейкой idcn_36
П

In [ ]:
from google.colab import files
edges.to_csv('edges.csv') 
files.download('edges.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
nodesus.to_csv('nodesus.csv') 
files.download('nodesus.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
downloaded = drive.CreateFile({'id':'1wFak7fSZXgTNbisRScpYw2EmEeAge961'})
downloaded.GetContentFile("nodesus_wide.csv",  mimetype='csv')
nds = pd.read_csv('nodesus_wide.csv')

In [20]:
res = nds.groupby("idcn")["idpr"].apply(list)
df = pd.DataFrame(res)
df.head()

,idpr
idcn,
0,"[4558511, 6398468, 6408431, 11102871, 11242417]"
1,"[3237298, 4707972, 5022317, 6330559, 6376319, ..."
2,"[1513033, 1547996, 3136956, 4642101, 7998422, ..."
3,[4618105]
4,"[1493650, 3035035, 4697503, 4737443, 4777348, ..."


In [21]:
res1 = nds.groupby("idcn")["is_correct"].apply(list)
df1 = pd.DataFrame(res1)
df1.head()

,is_correct
idcn,
0,"[0, 0, 0, 0, 0]"
1,"[1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 1, 0, 0, 1]"
3,[0]
4,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [22]:
res2 = nds.groupby("idcn")["is_hint_used"].apply(list)
df2 = pd.DataFrame(res2)
df2.head()

,is_hint_used
idcn,
0,"[1, 1, 1, 1, 1]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,[1]
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"


In [41]:
gnr = pd.merge(df,df1,on="idcn", how="inner")
gnr_1 = pd.merge(gnr,df2,on="idcn", how="inner")
gnr_2 = pd.merge(gnr_1,n,on="idcn", how="inner")
gnr_2.head()

,idcn,idpr,is_correct,is_hint_used,difficulty,learning_stage
0,0,"[4558511, 6398468, 6408431, 11102871, 11242417]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]",0.0,0.0
1,0,"[4558511, 6398468, 6408431, 11102871, 11242417]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]",0.0,0.0
2,0,"[4558511, 6398468, 6408431, 11102871, 11242417]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]",0.0,0.0
3,0,"[4558511, 6398468, 6408431, 11102871, 11242417]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]",0.0,0.0
4,0,"[4558511, 6398468, 6408431, 11102871, 11242417]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]",0.0,0.0


In [42]:
gnr_3 = df = pd.concat([gnr_2['idcn'],gnr_2['idpr'].apply(pd.Series)],axis=1)
gnr_4 = df = pd.concat([gnr_2['idcn'],gnr_2['is_correct'].apply(pd.Series)],axis=1)
gnr_5 = df = pd.concat([gnr_2['idcn'],gnr_2['is_hint_used'].apply(pd.Series)],axis=1)

In [ ]:
gn = pd.merge(gnr_3,gnr_4,on="idcn", how="inner")
gn_1 = pd.merge(gn,gnr_5,on="idcn", how="inner")
gn_2 = pd.merge(gn_1,n,on="idcn", how="inner")
gn_2.head()

In [ ]:
from google.colab import files
gn_2.to_csv('gn_2.csv') 
files.download('gn_2.csv')